# Dataset Construction

In this notebook we constructed the dataset for training by linking all these messy dataframes.

Some highlights:

* We simplified the problem by assuming a user would only consider buying staff he has bought before in 'prior' session
* For each user-product pair appeared before, generate statistical features about user, about product (including aisles and departments the products are associated with), and about this pairs.

In [1]:
import gc
import time
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from subprocess import check_output
from util import *

In [2]:
path_data = './data/'
priors, train, orders, products, aisles, departments, sample_submission = load_data(path_data)
products = products.merge(right=aisles,how='left',on='aisle_id')
products = products.merge(right=departments,how='left',on='department_id')
del aisles
del departments
gc.collect()

46

*priors_orders_detail* contains order entries included in *priors*, so there could exists products in *train* that is omitted.

In [3]:
priors_orders_detail = orders.merge(right=priors, how='inner', on='order_id')
priors_orders_detail['_user_buy_product_times'] = priors_orders_detail.groupby(['user_id', 'product_id']).cumcount() + 1
agg_dict = {'user_id':{'_prod_tot_cnts':'count'}, 
            'reordered':{'_prod_reorder_tot_cnts':'sum'}, 
            '_user_buy_product_times': {'_prod_buy_first_time_total_cnt':lambda x: sum(x==1),
                                        '_prod_buy_second_time_total_cnt':lambda x: sum(x==2)}}
prd = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['product_id'], agg_dict)

add stats features begin ......
add stats features end ......
time lapsing 121.348473072 s 



In [4]:
list(priors_orders_detail.columns)

['order_id',
 'user_id',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order',
 'product_id',
 'add_to_cart_order',
 'reordered',
 '_user_buy_product_times']

In [5]:
prd['reorder_ratio'] = prd['_prod_reorder_tot_cnts']/prd['_prod_tot_cnts']
prd['_prod_reorder_prob'] = prd._prod_buy_second_time_total_cnt / prd._prod_buy_first_time_total_cnt
prd['_prod_reorder_times'] = 1 + prd._prod_reorder_tot_cnts / prd._prod_buy_first_time_total_cnt
prd = prd.merge(right=products,how='left',on='product_id')

grouped_aisle = prd.groupby('aisle_id')
grouped_department = prd.groupby('department_id')

*prd* contains 'product_id' of products in *priors* transaction record.

In [7]:
print 'processing aisle...'
aisle = pd.concat([grouped_aisle['_prod_tot_cnts'].sum().rename('_aisle_tot_cuts'),
           grouped_aisle['_prod_buy_first_time_total_cnt'].sum().rename('_aisle_buy_first_time_total_cnt'),
           grouped_aisle['_prod_buy_second_time_total_cnt'].sum().rename('_aisle_buy_second_time_total_cnt'),
           grouped_aisle['_prod_reorder_tot_cnts'].sum().rename('_aisle_reorder_tot_cnts')
           ],axis=1).reset_index()
aisle['_aisle_reorder_prob'] = aisle._aisle_buy_second_time_total_cnt / aisle._aisle_buy_first_time_total_cnt
aisle['_aisle_reorder_times'] = 1 + aisle._aisle_reorder_tot_cnts / aisle._aisle_buy_first_time_total_cnt
print 'processing departments...'
departments = pd.concat([grouped_department['_prod_tot_cnts'].sum().rename('_department_tot_cuts'),
           grouped_department['_prod_buy_first_time_total_cnt'].sum().rename('_department_buy_first_time_total_cnt'),
           grouped_department['_prod_buy_second_time_total_cnt'].sum().rename('_department_buy_second_time_total_cnt'),
           grouped_department['_prod_reorder_tot_cnts'].sum().rename('_department_reorder_tot_cnts')
           ],axis=1).reset_index()
departments['_department_reorder_prob'] = departments._department_buy_second_time_total_cnt / departments._department_buy_first_time_total_cnt
departments['_department_reorder_times'] = 1 + departments._department_reorder_tot_cnts / departments._department_buy_first_time_total_cnt


processing aisle...
processing departments...


In [12]:
print 'merging everything...'
prd=prd.merge(aisle, how='inner',on='aisle_id')
prd=prd.merge(departments,how='inner',on='department_id')
prd.head(2)

merging everything...


,product_id,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,reorder_ratio,_prod_reorder_prob,_prod_reorder_times,product_name,aisle_id,...,_aisle_buy_second_time_total_cnt,_aisle_reorder_tot_cnts,_aisle_reorder_prob,_aisle_reorder_times,_department_tot_cuts,_department_buy_first_time_total_cnt,_department_buy_second_time_total_cnt,_department_reorder_tot_cnts,_department_reorder_prob,_department_reorder_times
0,1,276,716,1852,1136.0,0.613391,0.385475,2.586592,Chocolate Sandwich Cookies,61,...,39667,128431.0,0.375514,2.215811,2887550,1229577,483357,1657973.0,0.393108,2.348409
1,78,3,8,11,3.0,0.272727,0.375000,1.375000,Nutter Butter Cookie Bites Go-Pak,61,...,39667,128431.0,0.375514,2.215811,2887550,1229577,483357,1657973.0,0.393108,2.348409


In [14]:
#Clean wasteful
del aisle
del departments
gc.collect()
prd = prd.drop(['product_name','aisle','department','aisle_id','department_id'],axis=1)
del grouped_aisle
del grouped_department
gc.collect()

Start generating features on each user

In [17]:
# _user_reorder_ratio: number of reorders / number of 'come back' order
# _user_total_products: total number of products this user had ever bought
# _user_distinct_products: total number of unique products this user had ever bought
# _user_average_product: average number of products this user makes in one order

agg_dict_2 = {'order_number':{'_user_total_orders':'max'},
              'days_since_prior_order':{'_user_sum_days_since_prior_order':'sum', 
                                        '_user_mean_days_since_prior_order': 'mean'}}
users = ka_add_groupby_features_1_vs_n(orders[orders.eval_set == 'prior'], ['user_id'], agg_dict_2)
user_id_group = priors_orders_detail.groupby('user_id')
user_total_products = user_id_group['product_id'].count().rename('_user_total_products')
user_distinct_products = user_id_group['product_id'].nunique().rename('_user_distinct_products')
user_reorder_ratio = (user_id_group['reordered'].sum()/priors_orders_detail[priors_orders_detail['order_number'] > 1].groupby('user_id')['order_number'].count()).rename('_user_reorder_ratio')
us = pd.concat([user_total_products,user_distinct_products,user_reorder_ratio], axis=1).reset_index()
users = users.merge(us, how='inner')
users['_user_average_product'] = 1.0* users._user_total_products / users._user_total_orders
del user_id_group
del user_total_products
del user_distinct_products
del user_reorder_ratio

add stats features begin ......
add stats features end ......
time lapsing 0.304536104202 s 



In [18]:
# _user_largest_order: The largest number of products this user ever made in one order
# _user_smallest_order: The opposite

temp = priors_orders_detail.groupby(['order_id', 'user_id'])['add_to_cart_order'].max()
temp = temp.reset_index()
temp=temp.groupby('user_id')['add_to_cart_order'].agg([np.max, np.min]).reset_index()
temp = temp.rename(columns={'amax':'_user_largest_order','amin':'_user_smallest_order'})
users = users.merge(temp,on='user_id')
del temp


In [19]:
users['_user_average_basket'] = users._user_total_products / users._user_total_orders
us = orders[orders.eval_set != "prior"][['user_id', 'order_id', 'eval_set', 'days_since_prior_order']]
us.rename(index=str, columns={'days_since_prior_order': 'time_since_last_order'}, inplace=True)
users = users.merge(us, how='inner')
del us
gc.collect()
users.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders,_user_total_products,_user_distinct_products,_user_reorder_ratio,_user_average_product,_user_largest_order,_user_smallest_order,_user_average_basket,order_id,eval_set,time_since_last_order
0,1,19.555555,176.0,10,59,18,0.759259,5.900000,9,4,5.900000,1187899,train,14.0
1,2,15.230769,198.0,14,195,102,0.510989,13.928571,26,5,13.928571,1492625,train,30.0
2,3,12.090909,133.0,12,88,33,0.705128,7.333333,11,5,7.333333,2774568,test,11.0
3,4,13.750000,55.0,5,18,17,0.071429,3.600000,7,2,3.600000,329954,test,30.0
4,5,13.333333,40.0,4,37,23,0.538462,9.250000,12,5,9.250000,2196797,train,6.0


In [20]:
#user_product pair stats features
agg_dict_4 = {'order_number':{'_up_order_count': 'count', 
                              '_up_first_order_number': 'min', 
                              '_up_last_order_number':'max'}, 
              'add_to_cart_order':{'_up_average_cart_position': 'mean'}}

user_product = ka_add_groupby_features_1_vs_n(df=priors_orders_detail, 
                                                      group_columns_list=['user_id', 'product_id'], 
                                                      agg_dict=agg_dict_4)
user_product = user_product.merge(prd, how='inner', on='product_id').merge(users, how='inner', on='user_id')
user_product.head()

add stats features begin ......
add stats features end ......
time lapsing 16.0542259216 s 



,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,_user_total_products,_user_distinct_products,_user_reorder_ratio,_user_average_product,_user_largest_order,_user_smallest_order,_user_average_basket,order_id,eval_set,time_since_last_order
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,59,18,0.759259,5.9,9,4,5.9,1187899,train,14.0
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,59,18,0.759259,5.9,9,4,5.9,1187899,train,14.0
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,59,18,0.759259,5.9,9,4,5.9,1187899,train,14.0
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,59,18,0.759259,5.9,9,4,5.9,1187899,train,14.0
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,59,18,0.759259,5.9,9,4,5.9,1187899,train,14.0


In [22]:
user_product['_up_order_rate'] = user_product._up_order_count / user_product._user_total_orders
user_product['_up_order_since_last_order'] = user_product._user_total_orders - user_product._up_last_order_number
user_product['_up_order_rate_since_first_order'] = user_product._up_order_count / (user_product._user_total_orders - user_product._up_first_order_number + 1)

In [23]:
user_product.shape

(13307953, 41)

In [27]:
# add user_id to train set
train = train.merge(right=orders[['order_id', 'user_id']], how='left', on='order_id')

# Add 'reordered' feature to 'data'. For all user-product pair, in 'train' df we should have reordered=1
# if this pair appears in the 'train' df, and should gives 'NaN' if it doesn't appear.

# Underlying assumption: a user won't buy new staff, which didn't appear in 'prior' session.

# *And this might be a bad assumption*

user_product = user_product.merge(train[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')

# release Memory
del train, prd, users, priors_orders_detail, orders
gc.collect()

378

In [28]:
user_product._prod_reorder_tot_cnts=user_product._prod_reorder_tot_cnts.astype(np.int32)
user_product.time_since_last_order = user_product.time_since_last_order.astype(np.int32)
user_product._user_sum_days_since_prior_order=user_product._user_sum_days_since_prior_order.astype(np.int32)
user_product._aisle_reorder_tot_cnts = user_product._aisle_reorder_tot_cnts.astype(np.int32)
user_product._department_reorder_tot_cnts = user_product._department_reorder_tot_cnts.astype(np.int32)

This data df contains information on all user-product pair that appeared before in the prior transaction record.

In [32]:
import sys
def compress(data):
    starting_size = sys.getsizeof(data)
    i = 0
    for c, dtype in zip(data.columns, data.dtypes):
        if 'int' in str(dtype):
            if min(data[c]) >=0:
                max_int =  max(data[c])
                if max_int <= 255:
                    data[c] = data[c].astype(np.uint8)
                elif max_int <= 65535:
                    data[c] = data[c].astype(np.uint16)
                elif max_int <= 4294967295:
                    data[c] = data[c].astype(np.uint32)
                i += 1
    print("Number of colums adjusted: {}\n".format(i))
    ## Changing known reorderd col to smaller int size
    data['reordered'] = np.nan_to_num(data['reordered']).astype(np.uint8)
    data['reordered'][data['reordered']==0] = np.nan
    print("Reduced size {:.2%}".format(float(sys.getsizeof(data))/float(starting_size)))
    return data
user_product = compress(user_product)

Number of colums adjusted: 26



/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Reduced size 75.75%


In [33]:
# save to a pickle file
# This is a 2.65GB large dataset...
user_product.to_pickle('user_product.pkl')

In [34]:
del user_product
gc.collect()

374